In [1]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2,FasterRCNN_ResNet50_FPN_V2_Weights
import torch
from torch import nn
import sys
from PIL import Image
sys.path.append('..')
import torch.nn.functional as F

import pickle

import copy

batch_data = pickle.load(open('sample_data/batch_data.pkl', 'rb'))

sample_x, sample_y = batch_data
# Use the Faster R-CNN model for object detection
model = fasterrcnn_resnet50_fpn_v2(weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)  # Load pretrained Faster R-CNN model
model = model.cuda()

# Freeze batchnorm layers
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        m.eval()
        m.weight.requires_grad = False
        m.bias.requires_grad = False

target_y = copy.deepcopy(sample_y)



for i in range(len(sample_y)):
    for k in sample_y[i].keys():
        sample_y[i][k] = sample_y[i][k].cuda()
        target_y[i][k] = target_y[i][k].cuda()
        if k == "labels":
            target_y[i][k] = torch.ones_like(sample_y[i][k])
            
sample_x,sample_y,target_y = sample_x[0],sample_y[0],target_y[0]

In [ ]:
sample_x

In [ ]:
sample_x.shape

In [ ]:
target_y

In [2]:
# before peforimg any attack, We need to define the loss function
# loss function takes only one argument, which is the batch of data
# We can define the loss function as follows
def loss_fn(model,batch):
    # Forward pass through the Faster R-CNN model
    # The model expects a list of tensors (each tensor is a dictionary with boxes and labels)
    # Here, we assume sample_y is already in the correct format as a list of dictionaries
    # To avoid effect batchnorm, we need to deepcopy the model

    # Note: Faster R-CNN model in PyTorch returns a dictionary of losses
    sample_x, sample_y = batch
    loss_dict = model([sample_x], [sample_y])
    
    # Combine all losses (classification + bbox regression)
    # losses = sum(loss for loss in loss_dict.values())

    losses = loss_dict['loss_classifier']
    return losses

from functools import partial
loss_fn = partial(loss_fn, model) # now loss_fn takes only one argument

#### Attack

In [3]:
from abe.algorithm.attack import FGSM, PGD, BIM, MIFGSM, TIFGSM, DIFGSM, SINIFGSM, SSA

In [4]:
# first we need to define the attack task
from abe.task import AttackTask
from abe.type import ModelType

task = AttackTask(loss_fn=loss_fn, model_type=ModelType.IMAGECLASSIFICATION, is_targeted=False) # for targeted attack, set is_targeted=True

# then we can define the attack algorithm

attack = BIM(task,eps=32/255, alpha=2/255, steps=16) # BIM attack

adversarial_x = attack([sample_x, sample_y]) # all attack algorithms take a batch of data as input

targeted_task = AttackTask(loss_fn=loss_fn, model_type=ModelType.IMAGECLASSIFICATION, is_targeted=True)

targeted_attack = BIM(targeted_task,eps=32/255, alpha=2/255, steps=16) # BIM attack with targeted_task

targeted_adversarial_x = targeted_attack([sample_x, target_y]) # for targeted attack, the second element of the batch should be the target label

In [5]:
# Now we can plot the adversarial example

from abe.metric.visualization import plot_adversarial_sample

In [ ]:
# To plot the adversarial example, we need to provide the original sample and the adversarial sample and forward function

def forward(batch):
    model.eval()
    x, _ = batch
    pred = model(x)
    model.train()
    threshold = 0.5
    scores = pred[0]['scores']
    boxes = pred[0]['boxes']
    labels = pred[0]['labels']
    keep = scores > threshold
    scores = scores[keep]
    boxes = boxes[keep]
    labels = labels[keep]
    return boxes, labels

# we should transform the input data to the correct format N, C, H, W


plot_adversarial_sample(forward,batch=[sample_x.unsqueeze(0),sample_y],adversarial_sample=adversarial_x.unsqueeze(0),model_type=ModelType.OBJECTDETECTION)

In [ ]:
plot_adversarial_sample(forward,batch=[sample_x.unsqueeze(0),sample_y],adversarial_sample=targeted_adversarial_x.unsqueeze(0),model_type=ModelType.OBJECTDETECTION)

#### Explanation

In [8]:
from abe.algorithm.explanation import AMPE,IG, FastIG, SaliencyMap, SmoothGradient, MFABA

In [ ]:
# first we need to define the explanation task
from abe.task import ExplanationTask

# Explanation task takes loss function and forward function as initialization arguments

def loss_fn(batch):
    # Forward pass through the Faster R-CNN model
    # The model expects a list of tensors (each tensor is a dictionary with boxes and labels)
    # Here, we assume sample_y is already in the correct format as a list of dictionaries
    # To avoid effect batchnorm, we need to deepcopy the model
    # Freeze batchnorm layers
    # Note: Faster R-CNN model in PyTorch returns a dictionary of losses
    sample_x, sample_y = batch
    loss_dict = model([sample_x], [sample_y])
    
    # Combine all losses (classification + bbox regression)
    # losses = sum(loss for loss in loss_dict.values())

    losses = loss_dict['loss_box_reg'] # use a specific box regression loss # not implemented in the current version, instead we use all box regression loss
    return losses

def forward(batch):
    return model(batch)

In [10]:
explanation_task = ExplanationTask(loss_fn=loss_fn, forward_fn=forward, model_type=ModelType.OBJECTDETECTION)

In [11]:
# then we can define the explanation algorithm

explanation = AMPE(explanation_task)

attribution = explanation([sample_x, sample_y])

In [ ]:
# Now we can plot the attribution map

from abe.metric.visualization import plot_explanation_heatmap

plot_explanation_heatmap(attribution, sample_x)